# Coordinate Ascent Variational Inference for Gaussian Mixture Models


**STATS 305C: Applied Statistics III**

_Stanford University. Spring, 2022._


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slinderman/stats305c/blob/master/notebooks/lecture10_cavi_gmm.ipynb)

In this notebook you'll practice deriving and implementing coordinate ascent variational inference (CAVI) for a model you now know and love, the Gaussian mixture model. We will focus on the simple case in which the covariances are known for each mixture component, but we'll give some pointers at the end as to how you could generalize this approach.

---




## Generative model
Assume each mixture component has identity covariance. Then the generative model is,

1. Sample parameters for each mixture component:
\begin{align}
\theta_k &\sim \mathcal{N}(0, \beta^{-1} I) \\
\end{align}
2. Sample mixture weights,
\begin{align}
\pi &\sim \mathrm{Dir}(\alpha) 
\end{align}
3. Sample mixture assignments for each data point,
\begin{align}
z_n &\sim \mathrm{Cat}(\pi)
\end{align}
4. Sample data points given parameters and assignments,
\begin{align}
x_n &\sim \mathcal{N}(\theta_{z_n}, I)
\end{align}

As we showed in class, you can write the log joint probability as,
\begin{align}
\log p(X, Z, \Theta, \pi) &= \sum_{n=1}^N \sum_{k=1}^K \left[ \mathbb{I}[z_n = k] \left(\log \mathcal{N}(x_n \mid \theta_k, I)  + \log \pi_k \right)  \right] + \sum_{k=1}^K [\log \mathcal{N}(\theta_k \mid 0, \beta^{-1} I)] + \log \mathrm{Dir}(\pi \mid \alpha) 
\end{align}
where we have used the shorthand $X = \{x_n\}_{n=1}^N$, $Z = \{z_n\}_{n=1}^N$, and $\Theta = \{\theta_k\}_{k=1}^K$. 


## Part 1: Derive the conditional distributions

To speed you along, we've given you the answers for this part. Double check that you understand how we arrived at them, then proceed to Part 1 where you'll derive the corresponding CAVI updates.

### Problem 1a: Conditional distribution of the mixture means
Derive $p(z_n \mid x_n, \pi, \{\theta_k\}_{k=1}^K)$

---

**Solution:** 
\begin{align}
p(z_n \mid x_n, \pi, \{\theta_k\}_{k=1}^K) 
&\propto p(z_n \mid \pi) p(x_n \mid \Theta, z_n) \\
&\propto \pi_{z_n} \mathcal{N}(x_n \mid \theta_{z_n}, I)
\end{align}

Normalizing,
\begin{align}
p(z_n = k \mid x_n, \pi, \{\theta_k\}_{k=1}^K) 
&= \frac{\pi_{k} \mathcal{N}(x_n \mid \theta_{k}, I)}{\sum_{j=1}^K \pi_{j} \mathcal{N}(x_n \mid \theta_{j}, I)}
\end{align}


---


### Problem 1b: Conditional distribution of the mixture weights
Derive $p(\pi \mid \{z_n\}_{n=1}^N, \alpha)$


---

**Solution:** 
\begin{align}
p(\pi \mid \{z_n\}_{n=1}^N, \alpha) 
&\propto p(\pi \mid \alpha) \prod_{n=1}^N p(z_n \mid \pi) \\
&\propto \mathrm{Dir}(\pi \mid \alpha) \prod_{n=1}^N \prod_{k=1}^K \pi_k^{\mathbb{I}[z_n = k]} \\
&= \mathrm{Dir}(\pi \mid [\alpha_1 + N_1, \ldots, \alpha_K + N_K])
\end{align}
where
\begin{align}
 N_k = \sum_{n=1}^N \mathbb{I}[z_n = k]
\end{align}

---

### Problem 1c: Conditional distribution of the mixture means
Derive $p(\theta_k \mid \{x_n, z_n\}_{n=1}^N, 0, \beta^{-1} I)$


---

**Solution:** 
\begin{align}
p(\theta_k \mid \{x_n, z_n\}_{n=1}^N, 0, \beta) 
&\propto p(\theta_k \mid \beta) \prod_{n=1}^N p(x_n \mid \theta_k)^{\mathbb{I}[z_n = k]} \\
&\propto \mathcal{N}(\theta_k \mid 0, \beta^{-1} I) \prod_{n=1}^N \mathcal{N}(x_n \mid \theta_k, I)^{\mathbb{I}[z_n = k]} \\
&\propto \exp \left\{-\frac{\beta + N_k}{2} \theta_k^\top  \theta_k + \theta_k^\top \left( \sum_{n=1}^N \mathbb{I}[z_n = k] x_n \right) \right\} \\
&\propto \mathcal{N}\left(\theta_k \, \bigg| \, \frac{\bar{x}_k}{\beta + N_k}, \, \frac{1}{\beta + N_k} I  \right)
\end{align}
where
\begin{align}
\bar{x}_k &= \sum_{n=1}^N \mathbb{I}[z_n = k] x_n \\
N_k &= \sum_{n=1}^N \mathbb{I}[z_n = k]
\end{align}

---

## Part 2: Derive the CAVI updates
For the Gaussian mixture model we will use a mean field posterior approximation, which assumes that the parameters and latent variables are all independent:
\begin{align}
q(Z, \Theta, \pi) &= \prod_{n=1}^N \left[q(z_n; \lambda_n^{(z)}) \right] \, \prod_{k=1}^K \left[q(\theta_k; \lambda_k^{(\theta)}) \right] \, q(\pi; \lambda^{(\pi)})
\end{align}

We will find the optimal variational approximation via coordinate ascent on the ELBO. Recall that the general form for a CAVI update is to set the mean field factor for one variable $q(\vartheta_j; \lambda_j)$ equal to,
\begin{align}
q(\vartheta_j; \lambda_j) \propto \exp \left\{\mathbb{E}_{q(\vartheta_{\neg j}; \lambda_{\neg j})}\left[ \log p(\vartheta_j \mid x, \vartheta_{\neg j}) \right] \right\}
\end{align}
or equivalently, 
\begin{align}
\log q(\vartheta_j; \lambda_j) &= \mathbb{E}_{q(\vartheta_{\neg j}; \lambda_{\neg j})}\left[ \log p(\vartheta_j \mid x, \vartheta_{\neg j}) \right] + c
\end{align}
For models like this one, which are built of exponential family distributions with conjugate priors, these CAVI updates will have simple closed form solutions.

In Problem 1, you already derived the conditional distributions. Now you just have to compute the expected log conditional densities, where the expectation is taken with respect to the other variables.

### Problem 2a: Derive the CAVI update for the mixture means

The mixture assignments are discrete variables $z_n \in \{1,\ldots,K\}$ so their variational posterior must be a discrete distribution; aka a categorical distribution:
\begin{align}
q(z_n; \lambda_n^{(z)}) &= \mathrm{Cat}(z_n ; \lambda_n^{(z)})
\end{align}
where 
\begin{align}
\lambda_n^{(z)} = \left[ \lambda_{n,1}^{(z)}, \ldots, \lambda_{n,K}^{(z)} \right]^\top
\end{align}
are the variational parameters. They must be non-negative and sum to one. These are equivalent to the _responsibilities_ from Week 4. 

Derive an expression for $\lambda_{n,k}^{(z)}$ in terms of $\mathbb{E}_{q(\pi)}[\log \pi_k]$ and $\mathbb{E}_{q(\theta_k)}[\log \mathcal{N}(x_n \mid \theta_k, I)]$

---

_Your answer here_

---

### Problem 2b: Derive the CAVI update for the mixture weights

Show that 
\begin{align}
q(\pi; \lambda^{(\pi)}) = \mathrm{Dir}(\pi ; \lambda^{(\pi)})
\end{align}
where $\lambda^{(\pi)} \in \mathbb{R}_+^{K}$ is a vector of posterior concentrations. 

Derive the optimal update for the variational parameters $\lambda^{(\pi)}$ in terms of $\alpha$ and $\lambda_{n,k}^{(z)}$, using the fact that
\begin{align}
\mathbb{E}_{q(z_n)}[\mathbb{I}[z_n = k]] &= \lambda_{n,k}^{(z)}.
\end{align}

---

_Your answer here_

---

### Problem 2c: Derive the CAVI updates for the mixture means

Show that the optimal $q(\theta_k; \lambda_k^{(\theta)})$ is a Gaussian distribution,
\begin{align}
q(\theta_k; \lambda_k^{(\theta)}) = \mathcal{N}(\theta_k; \mu_k, \Sigma_k)
\end{align}
with $\lambda_k^{(\theta)} = ({\mu}_k, {\Sigma}_k)$.

Derive the variational parameter updates in terms of,
\begin{align}
 \bar{x}_k &\triangleq \sum_{n=1}^N \mathbb{E}_{q(z_n)} \left[ \mathbb{I}[z_n = k] \cdot x_n \right] \\
 &= \sum_{n=1}^N \lambda_{n,k}^{(z)} x_n
\end{align}
and
\begin{align}
N_k &\triangleq \sum_{n=1}^N \mathbb{E}_{q(z_n)} \left[ \mathbb{I}[z_n = k] \right] \\
&= \sum_{n=1}^N \lambda_{n,k}^{(z)}.
\end{align}
as well as the prior parameter $\beta$.

---

_Your answer here_

---

### Problem 2d: Derive the Gaussian cross entropy

The _negative cross entropy_ between $q(x)$ and $p(x)$ is defined as,
\begin{align}
\mathbb{E}_{q(x)}[\log p(x)]
\end{align}

Since $q(\theta_k)$ is Gaussian, and since $\mathcal{N}(x_n \mid \theta_k, I) = \mathcal{N}(\theta_k \mid x_n, I)$, we now recognize the $\mathbb{E}_{q(\theta_k)}[\log \mathcal{N}(x_n \mid \theta_k, I)]$ term in our $q(z)$ update as the negative cross entropy between two multivariate normal distributions.

Show that the negative cross entropy between two multivariate normal distributions is,
\begin{align}
\mathbb{E}_{\mathcal{N}(x \mid \mu_1, \Sigma_1)}[\log \mathcal{N}(x \mid \mu_2, \Sigma_2)] 
&= 
\log \mathcal{N}(\mu_1 \mid \mu_2, \Sigma_2) -\tfrac{1}{2} \mathrm{Tr}(\Sigma_1 \Sigma_2^{-1})
\end{align}


---

_Your answer here_

---

## Part 3: Derive the ELBO


### Problem 3a: Break the ELBO into parts
The ELBO is generically written as,
\begin{align}
\mathcal{L}(\lambda) &=
\mathbb{E}_q [\log p(X, Z, \Theta, \pi) - \log q(Z, \Theta, \pi) ]\end{align}
Rewrite the ELBO in terms of the following quantities,
- $\mathbb{E}_{q(z_n)}[\mathbb{I}[z_n = k]] \, \mathbb{E}_{q(\theta_k)}[\log \mathcal{N}(x_n \mid \theta_k, I)]$
- $\mathbb{E}_{q(\pi)} [\mathrm{KL}(q(z_n) \, \| \, \pi)]$
- $\mathrm{KL}(q(\pi) \, \| \, p(\pi))$
- $\mathrm{KL}(q(\theta_k) \, \| \, p(\theta_k))$

---

_Your answer here_

---

### Problem 3b: Derive closed form expressions for each term in the ELBO

Find closed form expressions for each term from Problem 3a.

---

_Your answer here_

---

## Problem 4: Implementation

In [ ]:
import torch
from torch.distributions import Dirichlet, MultivariateNormal, Categorical

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

from tqdm.auto import trange

In [ ]:
#@title Helper function to draw ellipse
def confidence_ellipse(mean, cov, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Modified from: https://matplotlib.org/3.5.0/gallery/\
        statistics/confidence_ellipse.html
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    mean: vector-like, shape (n,)
        Mean vector.
        
    cov : matrix-like, shape (n, n)
        Covariance matrix.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    # compute the 2D covariance ellipse
    pearson = cov[0, 1] / torch.sqrt(cov[0, 0] * cov[1, 1])
    ell_radius_x = torch.sqrt(1 + pearson)
    ell_radius_y = torch.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), 
                      width=ell_radius_x * 2, 
                      height=ell_radius_y * 2,
                      facecolor=facecolor, 
                      **kwargs)

    # Calculating the standard deviation
    # the square root of the variance and multiplying
    # with the given number of standard deviations.
    scale = torch.sqrt(torch.diag(cov) * n_std)
    
    # Transform the ellipse by rotating, scaling, and translating
    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(*scale) \
        .translate(*mean)
    ellipse.set_transform(transf + ax.transData)

    # Add the patch to the axis
    return ax.add_patch(ellipse)

## Write some helper fucntions for KL divergences and cross entropies


In [ ]:
def dirichlet_expected_log(dirichlet):
    """Helper function to compute expected log under Dirichlet distribution.

    Args:
        dirichlet: A torch.distributions.Dirichlet object with a batch shape of
            (...,) and a event shape of (K,).

    Returns:
        (...,K) tensor of expected logs, E[\log \pi], under the Dirichlet.
    """
    concentration = dirichlet.concentration
    return torch.special.digamma(concentration) - \
           torch.special.digamma(concentration.sum(dim=-1, keepdims=True))


def dirichlet_log_normalizer(concentration):
    """Compute the log normalizing constant of a Dirichlet distribution with
    the specificed concentration.

    Args:
        concentration: (...,K) tensor of concentration parameters

    Returns:
        (...,) batch of log normalizers
    """
    return torch.special.gammaln(concentration).sum(dim=-1) - \
        torch.special.gammaln(concentration.sum(dim=-1))

def dirichlet_kl(q, p):
    """Compute the KL divergence between two Dirichlet disdtributions

    Args:
        q: A torch.distributions.Dirichlet object
        p: A torch.distributions.Dirichlet object over the same domain

    Returns:
        A (batch of) KL divergence(s) between q and p.
    """
    kl = -dirichlet_log_normalizer(q.concentration)
    kl += dirichlet_log_normalizer(p.concentration)
    kl += torch.sum((q.concentration - p.concentration) * \
                    dirichlet_expected_log(q), dim=-1)
    return kl


def gaussian_neg_cross_entropy(q, p):
    """Compute the negative cross entropy between two Gaussian distributions,
        
        -E_{q(x | \mu_q, \Sigma_q)}[\log p(x | \mu_p, \Sigma_p)]

    Args:
        q: A torch.distributions.MultivariateNormal object
        p: A torch.distributions.MultivariateNormal object

    Returns:
        A (batch of) cross entropy(ies) between q and p.
    """
    mu_q = q.mean
    Sigma_q = q.covariance_matrix
    mu_p = p.mean
    Sigma_p = p.covariance_matrix

    # Compute the multivariate normal cross entropy
    return p.log_prob(mu_q) - 0.5 * torch.diagonal(
        torch.linalg.solve(Sigma_p, Sigma_q), dim1=-2, dim2=-1).sum(dim=-1)


def gaussian_kl(q, p):
    """Compute the KL divergence between two Gaussian distributions,
        
        E_{q(x | \mu_q, \Sigma_q)}[\log q(x | \mu_q, \Sigma_q) 
                                  - \log p(x | \mu_p, \Sigma_p)]

    Notes:
        This function is not at all optimized for speed. A better implementation
        would use the Cholesky of the covariance matrix rather than explicitly
        inverting it. C.f. https://github.com/tensorflow/probability/blob/main/
        tensorflow_probability/python/distributions/mvn_linear_operator.py#L327

    Args:
        q: A torch.distributions.MultivariateNormal object
        p: A torch.distributions.MultivariateNormal object

    Returns:
        A (batch of) cross entropy(ies) between q and p.
    """
    mu_q = q.mean
    Sigma_q = q.covariance_matrix
    mu_p = p.mean
    Sigma_p = p.covariance_matrix
    Sigma_p_inv = torch.inverse(Sigma_p)
    dim = mu_q.shape[-1]

    kl = 0.5 * (torch.logdet(Sigma_p) - torch.logdet(Sigma_q))
    kl -= 0.5 * dim
    kl += 0.5 * torch.diagonal(
        Sigma_p_inv @ Sigma_q, dim1=-2, dim2=-1).sum(dim=-1)
    kl += 0.5 * torch.einsum('...i,...ij,...j->...', 
                             mu_p - mu_q, Sigma_p_inv, mu_p - mu_q)
    return kl

In [ ]:
# These functions should broadcast appropriately
p = torch.distributions.MultivariateNormal(torch.zeros(5, 1, 2), torch.eye(2))
q = torch.distributions.MultivariateNormal(torch.zeros((1, 10, 2)), 
                                           2 * torch.eye(2).repeat(1, 10, 1, 1))
out = gaussian_kl(q, p)
out

In [ ]:
# Test our implementations against tensorflow probability
import jax.numpy as jnp

from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions

# Code to suppress stupid TFP warnings
import logging
logger = logging.getLogger()
class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()
logger.addFilter(CheckTypesFilter())

# Test Gaussian KL
p = MultivariateNormal(torch.zeros(2), torch.eye(2))
q = MultivariateNormal(torch.ones(2), 2 * torch.eye(2))
print("my kl:  ", gaussian_kl(q, p).numpy())
p2 = tfd.MultivariateNormalFullCovariance(jnp.zeros(2), jnp.eye(2))
q2 = tfd.MultivariateNormalFullCovariance(jnp.ones(2), 2 * jnp.eye(2))
print("tfp kl: ", q2.kl_divergence(p2))

# Test Gaussian cross entropy
p = MultivariateNormal(torch.zeros(2), torch.eye(2))
q = MultivariateNormal(torch.ones(2), 2 * torch.eye(2))
print("my ce:  ", gaussian_neg_cross_entropy(q, p).numpy())
p2 = tfd.MultivariateNormalFullCovariance(jnp.zeros(2), jnp.eye(2))
q2 = tfd.MultivariateNormalFullCovariance(jnp.ones(2), 2 * jnp.eye(2))
print("tfp ce: ", -q2.cross_entropy(p2))

# Test Dirichlet KL
p = Dirichlet(torch.ones(2))
q = Dirichlet(2 * torch.ones(2))
print("my kl:  ", dirichlet_kl(q, p).numpy())
p2 = tfd.Dirichlet(jnp.ones(2))
q2 = tfd.Dirichlet(2 * jnp.ones(2))
print("tfp kl: ", q2.kl_divergence(p2))


### Problem 4a: Implement CAVI updates


In [ ]:
def cavi_step_z(data, q_pi, q_theta):
    """
    Perform a CAVI update of q(z) given the data, q(\pi), and q(\theta)

    Args:
        data: (N, D) tensor where each row is a data point
        q_pi: td.Dirichlet posterior distribution over the mixture weights
        q_theta: td.MultivariateNormal posterior distribution over the set of
            mixture means. I.e. `q_theta.mean.shape == (K, D)` and 
            `q_theta.covariance_matrix.shape == (K, D, D)` where K is the 
            number of mixture components.

    Returns:
        q_z: a Categorical distribution over a batch of N mixture assignments.
    """
    ###
    # Your code here
    ##
    return Categorical(logits=...)
    

def cavi_step_pi(q_z, alpha):
    """
    Performa CAVI update of q(\pi) given q(z) and alpha.

    Args:
        q_z: Categorical posterior distribution over mixture assignments
        alpha: scalar (or shape (K,) tensor of) prior concentration(s)

    Returns:
        q_pi: Dirichlet posterior distribution over mixture weights
    """
    ###
    # Your code here
    ##
    return Dirichlet(...)


def cavi_step_theta(data, q_z, beta):
    """
    Performa CAVI update of q(\pi) given q(z) and alpha.

    Args:
        q_z: Categorical posterior distribution over mixture assignments
        alpha: scalar (or shape (K,) tensor of) prior concentration(s)

    Returns:
        q_pi: Dirichlet posterior distribution over mixture weights
    """
    ###
    # Your code here
    ##
    return MultivariateNormal(..., ...)
    

### Problem 4b: Implement the ELBO

In [ ]:
def elbo(data, q_z, q_pi, q_theta, alpha, beta):
    """Compute the evidence lower bound.
    """
    E_z = q_z.probs
    N, D = data.shape
    K = q_pi.concentration.shape[-1]

    ###
    # Your code here
    ##
    
    return elbo

### Put it all together

In [ ]:
def cavi(data, 
         num_components=2, 
         num_iter=100, 
         tol=1e-5, 
         alpha=20.0, 
         beta=1.0,
         seed=305 + ord('c'),
        ):
    """Run coordinate ascent VI for the Gaussian Mixture Model.
    
    """
    data = data.type(torch.float)
    N, D = data.shape
    K = num_components      # short hand
    
    # Initialize the clusters randomly
    torch.manual_seed(seed)
    clusters = Categorical(logits=torch.zeros(K)).sample((N,))
    q_pi = Dirichlet(alpha * torch.ones(K))
    q_theta = MultivariateNormal(
        torch.row_stack([data[clusters == k].mean(axis=0) for k in range(K)]),
        torch.eye(D).repeat(K, 1, 1))
    q_z = Categorical(logits=torch.zeros((N, K)))
    
    # Run CAVI
    elbos = [elbo(data, q_z, q_pi, q_theta, alpha, beta)]
    for itr in trange(num_iter):
        # Update variational factors one at a time
        q_z = cavi_step_z(data, q_pi, q_theta)
        q_pi = cavi_step_pi(q_z, alpha)
        q_theta = cavi_step_theta(data, q_z, beta)
        
        # Compute the ELBO
        elbos.append(elbo(data, q_z, q_pi, q_theta, alpha, beta))
        
        # Check for convergence
        if elbos[-1] - elbos[-2] < -1e-4:
            raise Exception("ELBO is going down!")
        elif elbos[-1] - elbos[-2] < tol:
            print("Converged!")
            break
        
    return torch.tensor(elbos), (q_z, q_pi, q_theta)
        

### Test on synthetic data

In [ ]:
# Generate synthetic data points   
torch.manual_seed(305 + ord('c'))
K = 3
D = 2
N = 60
true_thetas = torch.Tensor([[2, 2], [5, 5], [8, 8]])
X = MultivariateNormal(
    true_thetas, torch.eye(D)).sample((N // K,)).reshape(-1, D)

# Run the CAVI algorithm
elbos, (q_z, q_pi, q_theta) = \
    cavi(X, 
         num_components=K, 
         alpha=torch.ones(K),
         beta=1.0)
    
# Print the results  
for k in range(K):
    print("Cluster ", k, ":")
    print("\t E[mu_k]:   ", q_theta.mean[k,:])
    print("\t Cov[mu_k]: ", q_theta.covariance_matrix[k,:,:])
    print("\t E[pi_k]:   ", q_pi.mean[k])
    print("")

# Plot the log probabilities over EM iterations
plt.figure()
plt.plot(elbos[1:])
plt.xlabel("CAVI iteration")
plt.ylabel("ELBO")

# create a second figure to plot the clustered data
fig, ax = plt.subplots(figsize=(6, 6))

# plot scatter 
colors = ["red", "blue", "green"]
z_hat = torch.argmax(q_z.probs, 1)

for k in range(K):
    ax.plot(X[z_hat==k, 0], X[z_hat==k, 1], c=colors[k], ls='', marker='.')
    ax.plot(q_theta.mean[k, 0], q_theta.mean[k, 1], c=colors[k], 
            mec='k', marker='o', ms=8)


## Conclusion

Truth be told, I spent about an hour this morning debugging my CAVI implementation until I found a ridiculously simple bug! The code was still returning sensible results, but the ELBO was going down. I thought for sure it was a bug in my ELBO calculation (because that's where it usually is), but this time it was in my update for $q(\theta_k)$. In the end, I guess checking if the ELBO decreased was a useful debugging tool!